In [45]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
df = pd.read_csv('NP.csv')
df

,Date,Price,Grid load forecast,Wind power forecast
0,2013-01-01 00:00:00,31.05,42497.0,2798.0
1,2013-01-01 01:00:00,30.47,41463.0,2417.0
2,2013-01-01 02:00:00,28.92,40812.0,2036.0
3,2013-01-01 03:00:00,27.88,40246.0,1706.0
4,2013-01-01 04:00:00,26.96,40088.0,1427.0
...,...,...,...,...
52411,2018-12-24 19:00:00,50.72,52591.0,2544.0
52412,2018-12-24 20:00:00,49.86,51787.0,2785.0
52413,2018-12-24 21:00:00,49.09,51488.0,2919.0
52414,2018-12-24 22:00:00,49.02,50928.0,3119.0


# imbalance data

In [83]:
df1 = pd.read_csv('merged_2018.csv')
df2 = pd.read_csv('merged_2019.csv')
df3 = pd.read_csv('merged_2020.csv')
df4 = pd.read_csv('merged_2021.csv')

In [106]:
df5 = pd.concat([df1,df2,df3,df4]).reset_index(drop=True)
df5

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
0,2018-11-02 00:00:00,40.20,-260.77,30177,27902,28100,28048,0,2103,2994,28525,0,2957,3826
1,2018-11-02 00:30:00,47.65,-188.43,30487,27756,28282,27997,0,2325,3055,28488,0,2957,3826
2,2018-11-02 01:00:00,50.93,-73.84,29433,27241,28458,28421,0,2201,3063,28450,0,2886,3934
3,2018-11-02 01:30:00,50.77,-61.50,28317,26610,28477,28435,0,2252,2987,28395,0,2886,3934
4,2018-11-02 02:00:00,52.34,-78.26,27462,26301,28376,28369,0,2002,3050,28064,0,2847,4055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43723,2021-04-30 21:30:00,120.00,224.06,32721,31810,31881,31739,2,297,500,28934,0,3752,1228
43724,2021-04-30 22:00:00,135.00,315.67,31634,30150,30797,30733,1,323,468,28125,0,4152,1234
43725,2021-04-30 22:30:00,90.00,165.25,30513,28291,29688,29306,0,345,442,27377,0,4152,1234
43726,2021-04-30 23:00:00,85.00,261.71,29152,26626,28293,27831,1,349,400,24024,0,3921,1203


In [107]:
df5['HAimbalanceVolume'] = df5['HAimbalanceVolume'].shift(2)
df5['HAload'] = df5['HAload'].shift(2)
df5['HAdemand'] = df5['HAdemand'].shift(2)
df5['HAsolar'] = df5['HAsolar'].shift(2)
df5['HAwindOff'] = df5['HAwindOff'].shift(2)
df5['HAwindOn'] = df5['HAwindOn'].shift(2)

In [115]:
df6 = df5[2:].reset_index(drop=True)
df6

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
0,2018-11-02 01:00:00,50.93,-260.77,30177.0,27241,28100.0,28421,0.0,2103.0,2994.0,28450,0,2886,3934
1,2018-11-02 01:30:00,50.77,-188.43,30487.0,26610,28282.0,28435,0.0,2325.0,3055.0,28395,0,2886,3934
2,2018-11-02 02:00:00,52.34,-73.84,29433.0,26301,28458.0,28369,0.0,2201.0,3063.0,28064,0,2847,4055
3,2018-11-02 02:30:00,51.99,-61.50,28317.0,26160,28477.0,28446,0.0,2252.0,2987.0,27828,0,2847,4055
4,2018-11-02 03:00:00,47.60,-78.26,27462.0,25853,28376.0,28312,0.0,2002.0,3050.0,27467,0,2757,4064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43721,2021-04-30 21:30:00,120.00,278.99,34410.0,31810,33356.0,31739,1.0,283.0,573.0,28934,0,3752,1228
43722,2021-04-30 22:00:00,135.00,384.94,33750.0,30150,32882.0,30733,1.0,256.0,528.0,28125,0,4152,1234
43723,2021-04-30 22:30:00,90.00,224.06,32721.0,28291,31881.0,29306,2.0,297.0,500.0,27377,0,4152,1234
43724,2021-04-30 23:00:00,85.00,315.67,31634.0,26626,30797.0,27831,1.0,323.0,468.0,24024,0,3921,1203


In [117]:
cols = ['HAload','HAdemand','HAsolar','HAwindOff','HAwindOn']
df6[cols] = df6[cols].applymap(np.int64)

In [118]:
df6[::2].to_csv('imbalanceData_final.csv',index=False)

# DA auction data

In [145]:
lF1 = df6['loadForecast'][::2].reset_index(drop=True)
lF2 = df6['loadForecast'][1::2].reset_index(drop=True)
lF = lF1+lF2

dF1 = df6['demandForecast'][::2].reset_index(drop=True)
dF2 = df6['demandForecast'][1::2].reset_index(drop=True)
dF = dF1+dF2

aF1 = df6['aggrForecast'][::2].reset_index(drop=True)
aF2 = df6['aggrForecast'][1::2].reset_index(drop=True)
aF = aF1+aF2

sF1 = df6['solarForecast'][::2].reset_index(drop=True)
sF2 = df6['solarForecast'][1::2].reset_index(drop=True)
sF = sF1+sF2

wF1 = df6['windOffForecast'][::2].reset_index(drop=True)
wF2 = df6['windOffForecast'][1::2].reset_index(drop=True)
woF = wF1+wF2

wF3 = df6['windOnForecast'][::2].reset_index(drop=True)
wF4 = df6['windOnForecast'][1::2].reset_index(drop=True)
wnF = wF3+wF4

In [132]:
pr18 = pd.read_csv('N2EX DA Auction Price/DA_HourlyPrice_2018.csv')
pr19 = pd.read_csv('N2EX DA Auction Price/DA_HourlyPrice_2019.csv')
pr20 = pd.read_csv('N2EX DA Auction Price/DA_HourlyPrice_2020.csv')
pr21 = pd.read_csv('N2EX DA Auction Price/DA_HourlyPrice_2021.csv')

In [133]:
pr = pd.concat([pr18,pr19,pr20,pr21]).reset_index(drop=True)
pr

,Unnamed: 0,Hours,CET/CEST time,UK
0,01-01-2018,23 - 00,00 - 01,"42,94"
1,01-01-2018,00 - 01,01 - 02,"47,14"
2,01-01-2018,01 - 02,02 - 03,"54,60"
3,01-01-2018,02 - 03,03 - 04,"46,63"
4,01-01-2018,03 - 04,04 - 05,"39,90"
...,...,...,...,...
31004,15-07-2021,18 - 19,19 - 20,"108,83"
31005,15-07-2021,19 - 20,20 - 21,"101,29"
31006,15-07-2021,20 - 21,21 - 22,"91,11"
31007,15-07-2021,21 - 22,22 - 23,"91,83"


In [143]:
pr1 = pr.iloc[7322:29185].reset_index(drop=True)
pr1

,Unnamed: 0,Hours,CET/CEST time,UK
0,02-11-2018,00 - 01,01 - 02,"52,24"
1,02-11-2018,01 - 02,02 - 03,"50,71"
2,02-11-2018,02 - 03,03 - 04,"50,56"
3,02-11-2018,03 - 04,04 - 05,"49,52"
4,02-11-2018,04 - 05,05 - 06,"49,96"
...,...,...,...,...
21858,30-04-2021,18 - 19,19 - 20,"119,08"
21859,30-04-2021,19 - 20,20 - 21,"106,00"
21860,30-04-2021,20 - 21,21 - 22,"96,67"
21861,30-04-2021,21 - 22,22 - 23,"90,00"


In [144]:
pr1['UK'] = pr1['UK'].str.replace(',','.')
pr1['auctionPrice'] = pd.to_numeric(pr1['UK'])
pr1

,Unnamed: 0,Hours,CET/CEST time,UK,auctionPrice
0,02-11-2018,00 - 01,01 - 02,52.24,52.24
1,02-11-2018,01 - 02,02 - 03,50.71,50.71
2,02-11-2018,02 - 03,03 - 04,50.56,50.56
3,02-11-2018,03 - 04,04 - 05,49.52,49.52
4,02-11-2018,04 - 05,05 - 06,49.96,49.96
...,...,...,...,...,...
21858,30-04-2021,18 - 19,19 - 20,119.08,119.08
21859,30-04-2021,19 - 20,20 - 21,106.00,106.00
21860,30-04-2021,20 - 21,21 - 22,96.67,96.67
21861,30-04-2021,21 - 22,22 - 23,90.00,90.00


In [149]:
date = df6[::2]['datetime'].reset_index(drop=True)
pr2 = pd.concat([date, pr1['auctionPrice'], lF, dF, aF, sF, woF, wnF],axis=1)
pr2

,datetime,auctionPrice,loadForecast,demandForecast,aggrForecast,solarForecast,windOffForecast,windOnForecast
0,2018-11-02 01:00:00,52.24,53851,56856,56845,0,5772,7868
1,2018-11-02 02:00:00,50.71,52461,56815,55892,0,5694,8110
2,2018-11-02 03:00:00,50.56,51589,56089,54787,0,5514,8128
3,2018-11-02 04:00:00,49.52,52108,55422,54548,0,5344,7848
4,2018-11-02 05:00:00,49.96,56408,56967,56399,17,5032,7368
...,...,...,...,...,...,...,...,...
21858,2021-04-30 19:00:00,119.08,68605,69325,61658,875,5206,2978
21859,2021-04-30 20:00:00,106.00,67759,68815,60080,32,6150,2580
21860,2021-04-30 21:00:00,96.67,64879,65068,58365,0,7504,2456
21861,2021-04-30 22:00:00,90.00,58441,60039,55502,0,8304,2468


In [150]:
pr2.to_csv('auctionData_final.csv',index=False)

# 2020

In [36]:
df1 = pd.read_csv('BMRS_2020/Imbalance_Price_2020.csv')
df2 = pd.read_csv('BMRS_2020/Imbalance_Volume_2020.csv')

df3 = pd.read_csv('BMRS_2020/Actual_Load_2020.csv')
df4 = pd.read_csv('BMRS_2020/DA_Load_Forecast_2020.csv')

df5 = pd.read_csv('BMRS_2020/Demand_Outturn_2020.csv')
df6 = pd.read_csv('BMRS_2020/DA_Demand_Forecast_2020.csv')

df7 = pd.read_csv('BMRS_2020/Actual_Solar_Generation_2020.csv')
df8 = pd.read_csv('BMRS_2020/Actual_Wind_Offshore_Generation_2020.csv')
df9 = pd.read_csv('BMRS_2020/Actual_Wind_Onshore_Generation_2020.csv')

df10 = pd.read_csv('BMRS_2020/DA_Aggregated_Generation_Forecast_2020.csv')
df11 = pd.read_csv('BMRS_2020/DA_Solar_Generation_Forecast_2020.csv')
df12 = pd.read_csv('BMRS_2020/DA_Wind_Offshore_Generation_Forecast_2020.csv')
df13 = pd.read_csv('BMRS_2020/DA_Wind_Onshore_Generation_Forecast_2020.csv')

In [86]:
df7.drop_duplicates(subset=['local_datetime', 'settlementPeriod'])

,local_datetime,settlementPeriod,quantity
0,2020-01-01 00:00:00+00:00,1,0.0
1,2020-01-01 00:30:00+00:00,2,0.0
2,2020-01-01 01:00:00+00:00,3,0.0
3,2020-01-01 01:30:00+00:00,4,0.0
4,2020-01-01 02:00:00+00:00,5,0.0
...,...,...,...
17563,2020-12-31 21:30:00+00:00,44,0.0
17564,2020-12-31 22:00:00+00:00,45,0.0
17565,2020-12-31 22:30:00+00:00,46,0.0
17566,2020-12-31 23:00:00+00:00,47,0.0


In [91]:
dfs = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13]
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['local_datetime','settlementPeriod'],how='outer'), dfs)
df_merged

,local_datetime,settlementPeriod,imbalancePriceAmountGBP,imbalanceQuantityMAW,quantity_x,quantity_y,demand_x,demand_y,quantity_x,quantity_y,quantity_x,quantity_y,quantity_x,quantity_y,quantity
0,2020-01-01 00:00:00+00:00,1,50.90000,54.3365,30675.0,27684,27073.0,27395.0,0.0,2335.105,3017.245,24897,0,2843.181,3024.238
1,2020-01-01 00:30:00+00:00,2,51.00000,194.7133,31260.0,28315,27612.0,28042.0,0.0,2541.627,3015.961,24701,0,2843.181,3024.238
2,2020-01-01 01:00:00+00:00,3,29.37006,-71.4292,30972.0,27745,27111.0,27890.0,0.0,2585.939,2967.431,24307,0,2696.680,2936.507
3,2020-01-01 01:30:00+00:00,4,28.52000,-122.2322,30110.0,26773,26311.0,27009.0,0.0,2507.583,2881.878,23833,0,2696.680,2936.507
4,2020-01-01 02:00:00+00:00,5,25.78000,-90.9042,29344.0,26099,25693.0,26332.0,0.0,2437.297,2781.775,23557,0,2524.649,2935.947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17563,2020-12-31 21:30:00+00:00,44,74.90000,537.2756,37242.0,32187,32781.0,32896.0,0.0,1853.476,3286.864,29136,0,2829.566,3086.609
17564,2020-12-31 22:00:00+00:00,45,73.15000,230.4028,36055.0,31123,31592.0,31500.0,0.0,1779.019,3317.274,28398,0,2886.828,3116.990
17565,2020-12-31 22:30:00+00:00,46,67.00000,50.3184,35252.0,30231,30802.0,30934.0,0.0,1848.772,3289.758,27126,0,2886.828,3116.990
17566,2020-12-31 23:00:00+00:00,47,72.00000,408.3631,34224.0,29027,29878.0,29918.0,0.0,1883.662,3155.358,24838,0,2990.172,3105.867


In [92]:
df_merged.columns = ['datetime','settlementPeriod','imbalancePrice','HAimbalanceVolume',
                     'HAload','loadForecast','HAdemand','demandForecast',
                     'HAsolar','HAwindOff','HAwindOn',
                     'aggrForecast','solarForecast','windOffForecast','windOnForecast']
df_merged['datetime'] = df_merged['datetime'].str[:-6]
df_merged.head()

,datetime,settlementPeriod,imbalancePrice,imbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
0,2020-01-01 00:00:00,1,50.90000,54.3365,30675.0,27684,27073.0,27395.0,0.0,2335.105,3017.245,24897,0,2843.181,3024.238
1,2020-01-01 00:30:00,2,51.00000,194.7133,31260.0,28315,27612.0,28042.0,0.0,2541.627,3015.961,24701,0,2843.181,3024.238
2,2020-01-01 01:00:00,3,29.37006,-71.4292,30972.0,27745,27111.0,27890.0,0.0,2585.939,2967.431,24307,0,2696.680,2936.507
3,2020-01-01 01:30:00,4,28.52000,-122.2322,30110.0,26773,26311.0,27009.0,0.0,2507.583,2881.878,23833,0,2696.680,2936.507
4,2020-01-01 02:00:00,5,25.78000,-90.9042,29344.0,26099,25693.0,26332.0,0.0,2437.297,2781.775,23557,0,2524.649,2935.947


In [156]:
df_merged.drop('settlementPeriod', axis=1, inplace=True)

In [157]:
df_merged.iloc[4224:4228]

,datetime,imbalancePrice,imbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
4224,2020-03-29 00:00:00,4.27,-879.8241,24260.0,23496,22613.0,22116.0,0.0,5683.107,4848.363,22157,0,8281.129,4967.828
4225,2020-03-29 00:30:00,4.27,-828.2884,24356.0,23495,22811.0,22360.0,0.0,203.167,2145.328,21976,0,8281.129,4967.828
4226,2020-03-29 02:00:00,2.77,-540.9614,23934.0,23136,22948.0,22705.0,0.0,5787.387,4653.959,21837,0,8172.215,4783.179
4227,2020-03-29 02:30:00,4.00,-272.9785,23675.0,22671,22761.0,22741.0,0.0,5445.079,4690.654,21778,0,8172.215,4783.179


In [161]:
def insert_row(idx, df, df_insert):
    return df.iloc[:idx,].append(df_insert).append(df.iloc[idx:,]).reset_index(drop = True)

aveT = df_merged.iloc[4225:4227].mean(axis=0)
ave = pd.DataFrame(aveT).transpose()
ave.insert(loc=0, column='datetime', value='2020-03-29 01:00:00')
inserted = insert_row(4226, df_merged, ave)
ave['datetime'] = '2020-03-29 01:30:00'
inserted = insert_row(4227, inserted, ave)

In [165]:
inserted.iloc[4225:4229]

,datetime,imbalancePrice,imbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
4225,2020-03-29 00:30:00,4.27,-828.2884,24356.0,23495.0,22811.0,22360.0,0.0,203.167,2145.3280,21976.0,0.0,8281.129,4967.8280
4226,2020-03-29 01:00:00,3.52,-684.6249,24145.0,23315.5,22879.5,22532.5,0.0,2995.277,3399.6435,21906.5,0.0,8226.672,4875.5035
4227,2020-03-29 01:30:00,3.52,-684.6249,24145.0,23315.5,22879.5,22532.5,0.0,2995.277,3399.6435,21906.5,0.0,8226.672,4875.5035
4228,2020-03-29 02:00:00,2.77,-540.9614,23934.0,23136.0,22948.0,22705.0,0.0,5787.387,4653.9590,21837.0,0.0,8172.215,4783.1790


In [163]:
inserted.iloc[14304:14311]

,datetime,imbalancePrice,imbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
14304,2020-10-25 00:00:00,64.00,420.7030,23009.0,21931.0,22798.0,22401.0,0.0,5373.415,4108.742,23602.0,0.0,8596.864,5696.236
14305,2020-10-25 00:30:00,-18.81,-685.8824,22187.0,21639.0,22201.0,21785.0,0.0,191.370,1910.845,23041.0,0.0,8596.864,5696.236
14306,2020-10-25 01:00:00,-18.78,-441.6049,21579.0,22498.0,22472.0,22135.0,0.0,190.002,1916.111,22806.0,0.0,8649.528,4890.292
14307,2020-10-25 01:30:00,-18.78,-431.3384,21695.0,22706.0,22584.0,22710.0,0.0,190.002,1916.111,22960.0,0.0,8649.528,4890.292
14308,2020-10-25 01:00:00,66.00,816.6748,22094.0,22198.0,21797.0,21741.0,0.0,187.435,1963.281,22970.0,0.0,8676.829,5183.471
14309,2020-10-25 01:30:00,-39.32,-1239.7079,21453.0,21591.0,21170.0,21320.0,0.0,187.435,1963.281,23084.0,0.0,8676.829,5183.471
14310,2020-10-25 02:00:00,-24.50,-479.1590,20002.0,21215.0,21810.0,21993.0,0.0,5451.169,3521.000,23147.0,0.0,8598.614,5505.488


In [167]:
aveT1 = inserted.iloc[14306:14308].mean(axis=0)
aveT2 = inserted.iloc[14308:14310].mean(axis=0)
ave1 = pd.DataFrame(aveT1).transpose()
ave2 = pd.DataFrame(aveT2).transpose()

ins2 = inserted.drop([14306,14307,14308,14309])

ave1.insert(loc=0, column='datetime', value='2020-10-25 01:00:00')
ins2 = insert_row(14306, ins2, ave1)
ave2.insert(loc=0, column='datetime', value='2020-10-25 01:30:00')
ins2 = insert_row(14307, ins2, ave2)

In [168]:
ins2.iloc[4225:4229]

,datetime,imbalancePrice,imbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
4225,2020-03-29 00:30:00,4.27,-828.2884,24356.0,23495.0,22811.0,22360.0,0.0,203.167,2145.3280,21976.0,0.0,8281.129,4967.8280
4226,2020-03-29 01:00:00,3.52,-684.6249,24145.0,23315.5,22879.5,22532.5,0.0,2995.277,3399.6435,21906.5,0.0,8226.672,4875.5035
4227,2020-03-29 01:30:00,3.52,-684.6249,24145.0,23315.5,22879.5,22532.5,0.0,2995.277,3399.6435,21906.5,0.0,8226.672,4875.5035
4228,2020-03-29 02:00:00,2.77,-540.9614,23934.0,23136.0,22948.0,22705.0,0.0,5787.387,4653.9590,21837.0,0.0,8172.215,4783.1790


In [170]:
ins2.iloc[14305:14309]

,datetime,imbalancePrice,imbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
14305,2020-10-25 00:30:00,-18.81,-685.88240,22187.0,21639.0,22201.0,21785.0,0.0,191.370,1910.845,23041.0,0.0,8596.864,5696.236
14306,2020-10-25 01:00:00,-18.78,-436.47165,21637.0,22602.0,22528.0,22422.5,0.0,190.002,1916.111,22883.0,0.0,8649.528,4890.292
14307,2020-10-25 01:30:00,13.34,-211.51655,21773.5,21894.5,21483.5,21530.5,0.0,187.435,1963.281,23027.0,0.0,8676.829,5183.471
14308,2020-10-25 02:00:00,-24.50,-479.15900,20002.0,21215.0,21810.0,21993.0,0.0,5451.169,3521.000,23147.0,0.0,8598.614,5505.488


In [173]:
decimals = pd.Series([2,2,0,0,0,0,0,0,0,0,0,0,0], index=['imbalancePrice','HAimbalanceVolume',
                                                         'HAload','loadForecast','HAdemand','demandForecast',
                                                         'HAsolar','HAwindOff','HAwindOn',
                                                         'aggrForecast','solarForecast','windOffForecast','windOnForecast'])
df_rounded = ins2.round(decimals)

In [ ]:
cols = ['HAload','loadForecast','HAdemand','demandForecast',
        'HAsolar','HAwindOff','HAwindOn',
        'aggrForecast','solarForecast','windOffForecast','windOnForecast']
df_rounded[cols] = df_rounded[cols].applymap(np.int64)

In [174]:
df_rounded.to_csv('merged_2020.csv',index=False)

# 2019

In [25]:
df1 = pd.read_csv('BMRS_2019/Imbalance_Price_2019.csv')
df2 = pd.read_csv('BMRS_2019/Imbalance_Volume_2019.csv')

df3 = pd.read_csv('BMRS_2019/Actual_Load_2019.csv')
df4 = pd.read_csv('BMRS_2019/DA_Load_Forecast_2019.csv')

df5 = pd.read_csv('BMRS_2019/Demand_Outturn_2019.csv')
df6 = pd.read_csv('BMRS_2019/DA_Demand_Forecast_2019.csv')

df7 = pd.read_csv('BMRS_2019/Actual_Solar_Generation_2019.csv')
df8 = pd.read_csv('BMRS_2019/Actual_Wind_Offshore_Generation_2019.csv')
df9 = pd.read_csv('BMRS_2019/Actual_Wind_Onshore_Generation_2019.csv')

df10 = pd.read_csv('BMRS_2019/DA_Aggregated_Generation_Forecast_2019.csv')
df11 = pd.read_csv('BMRS_2019/DA_Solar_Generation_Forecast_2019.csv')
df12 = pd.read_csv('BMRS_2019/DA_Wind_Offshore_Generation_Forecast_2019.csv')
df13 = pd.read_csv('BMRS_2019/DA_Wind_Onshore_Generation_Forecast_2019.csv')

In [24]:
df2.drop_duplicates(subset=['local_datetime', 'settlementPeriod'])

,local_datetime,settlementPeriod,imbalanceQuantityMAW
0,2019-01-01 00:00:00+00:00,1,-1058.3066
1,2019-01-01 00:30:00+00:00,2,-664.2875
2,2019-01-01 01:00:00+00:00,3,-1033.9092
3,2019-01-01 01:30:00+00:00,4,-1319.3434
4,2019-01-01 02:00:00+00:00,5,-1180.8583
...,...,...,...
17515,2019-12-31 21:30:00+00:00,44,65.1316
17516,2019-12-31 22:00:00+00:00,45,64.3746
17517,2019-12-31 22:30:00+00:00,46,-126.6860
17518,2019-12-31 23:00:00+00:00,47,-331.9134


In [26]:
dfs = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13]
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['local_datetime','settlementPeriod'],how='outer'), dfs)
df_merged

,local_datetime,settlementPeriod,imbalancePriceAmountGBP,imbalanceQuantityMAW,quantity_x,quantity_y,demand_x,demand_y,quantity_x,quantity_y,quantity_x,quantity_y,quantity_x,quantity_y,quantity
0,2019-01-01 00:00:00+00:00,1,15.00,-1058.3066,26723,27271,25183.0,25691.0,0,3141.711,5986.581,26378,0,3864.961,7805.757
1,2019-01-01 00:30:00+00:00,2,15.00,-664.2875,27458,27676,25633.0,26091.0,0,3253.887,5830.487,26313,0,3864.961,7805.757
2,2019-01-01 01:00:00+00:00,3,16.00,-1033.9092,27468,26848,25384.0,26013.0,0,3340.851,5900.198,26093,0,3968.607,7447.149
3,2019-01-01 01:30:00+00:00,4,16.00,-1319.3434,26447,25675,24456.0,25177.0,0,3213.702,5780.874,25738,0,3968.607,7447.149
4,2019-01-01 02:00:00+00:00,5,16.00,-1180.8583,24954,24773,24255.0,25035.0,0,3122.706,5605.429,25442,0,3975.696,7054.965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019-12-31 21:30:00+00:00,44,52.50,65.1316,33752,30182,30654.0,31237.0,0,1816.148,3437.526,30443,0,3192.447,3313.707
17516,2019-12-31 22:00:00+00:00,45,52.00,64.3746,31338,29438,29496.0,29894.0,0,1985.109,3582.422,30030,0,3293.298,3323.089
17517,2019-12-31 22:30:00+00:00,46,11.15,-126.6860,30591,28588,28694.0,28830.0,0,2177.970,3374.994,29190,0,3293.298,3323.089
17518,2019-12-31 23:00:00+00:00,47,12.00,-331.9134,30764,27568,27818.0,28229.0,0,2268.417,3227.063,27416,0,3252.237,3360.136


In [27]:
df_merged.columns = ['datetime','settlementPeriod','imbalancePrice','HAimbalanceVolume',
                     'HAload','loadForecast','HAdemand','demandForecast',
                     'HAsolar','HAwindOff','HAwindOn',
                     'aggrForecast','solarForecast','windOffForecast','windOnForecast']
df_merged['datetime'] = df_merged['datetime'].str[:-6]
df_merged.head()

,datetime,settlementPeriod,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
0,2019-01-01 00:00:00,1,15.0,-1058.3066,26723,27271,25183.0,25691.0,0,3141.711,5986.581,26378,0,3864.961,7805.757
1,2019-01-01 00:30:00,2,15.0,-664.2875,27458,27676,25633.0,26091.0,0,3253.887,5830.487,26313,0,3864.961,7805.757
2,2019-01-01 01:00:00,3,16.0,-1033.9092,27468,26848,25384.0,26013.0,0,3340.851,5900.198,26093,0,3968.607,7447.149
3,2019-01-01 01:30:00,4,16.0,-1319.3434,26447,25675,24456.0,25177.0,0,3213.702,5780.874,25738,0,3968.607,7447.149
4,2019-01-01 02:00:00,5,16.0,-1180.8583,24954,24773,24255.0,25035.0,0,3122.706,5605.429,25442,0,3975.696,7054.965


In [28]:
df_merged.drop('settlementPeriod', axis=1, inplace=True)

In [29]:
df_merged.iloc[4272:4276]

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
4272,2019-03-31 00:00:00,47.9000,398.9707,28666,25399,25570.0,25880.0,0,1238.315,1607.682,23913,0,2227.323,1370.499
4273,2019-03-31 00:30:00,48.1000,496.9227,28422,25280,26111.0,26381.0,0,59.296,795.831,23905,0,2227.323,1370.499
4274,2019-03-31 02:00:00,47.9381,411.8521,27998,24853,25943.0,26380.0,0,1440.454,1551.774,23633,0,2482.908,1328.463
4275,2019-03-31 02:30:00,47.4000,116.3980,26981,24269,25107.0,25704.0,0,1576.093,1507.938,23626,0,2482.908,1328.463


In [30]:
def insert_row(idx, df, df_insert):
    return df.iloc[:idx,].append(df_insert).append(df.iloc[idx:,]).reset_index(drop = True)

aveT = df_merged.iloc[4273:4275].mean(axis=0)
ave = pd.DataFrame(aveT).transpose()
ave.insert(loc=0, column='datetime', value='2019-03-31 01:00:00')
inserted = insert_row(4274, df_merged, ave)
ave['datetime'] = '2019-03-31 01:30:00'
inserted = insert_row(4275, inserted, ave)

In [32]:
inserted.iloc[4273:4277]

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
4273,2019-03-31 00:30:00,48.10000,496.9227,28422.0,25280.0,26111.0,26381.0,0.0,59.296,795.8310,23905.0,0.0,2227.3230,1370.499
4274,2019-03-31 01:00:00,48.01905,454.3874,28210.0,25066.5,26027.0,26380.5,0.0,749.875,1173.8025,23769.0,0.0,2355.1155,1349.481
4275,2019-03-31 01:30:00,48.01905,454.3874,28210.0,25066.5,26027.0,26380.5,0.0,749.875,1173.8025,23769.0,0.0,2355.1155,1349.481
4276,2019-03-31 02:00:00,47.93810,411.8521,27998.0,24853.0,25943.0,26380.0,0.0,1440.454,1551.7740,23633.0,0.0,2482.9080,1328.463


In [33]:
inserted.iloc[14352:14359]

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
14352,2019-10-27 00:00:00,-60.00,-699.1362,25917.0,23522.0,24440.0,25147.0,0.0,5103.553,4486.048,25086.0,0.0,6400.315,6392.602
14353,2019-10-27 00:30:00,-68.97,-902.9365,25071.0,21718.0,23840.0,24475.0,0.0,152.484,1895.443,24888.0,0.0,6400.315,6392.602
14354,2019-10-27 01:00:00,-70.00,-930.8751,24434.0,23656.0,23635.0,24127.0,0.0,151.615,1930.092,24794.0,0.0,6495.704,6293.020
14355,2019-10-27 01:30:00,0.00,-684.3550,24975.0,23710.0,23789.0,24697.0,0.0,151.615,1930.092,24777.0,0.0,6495.704,6293.020
14356,2019-10-27 01:00:00,2.00,-946.1553,24389.0,23180.0,23652.0,24308.0,0.0,151.840,1961.328,24695.0,0.0,6496.542,6350.449
14357,2019-10-27 01:30:00,-60.00,-836.6718,23978.0,22710.0,23254.0,23855.0,0.0,151.840,1961.328,24679.0,0.0,6496.542,6350.449
14358,2019-10-27 02:00:00,0.00,-576.1649,22720.0,22445.0,22911.0,23512.0,0.0,4633.319,4009.640,24685.0,0.0,6411.804,6384.826


In [34]:
aveT1 = inserted.iloc[14354:14356].mean(axis=0)
aveT2 = inserted.iloc[14356:14358].mean(axis=0)
ave1 = pd.DataFrame(aveT1).transpose()
ave2 = pd.DataFrame(aveT2).transpose()

ins2 = inserted.drop([14354,14355,14356,14357])

ave1.insert(loc=0, column='datetime', value='2019-10-27 01:00:00')
ins2 = insert_row(14354, ins2, ave1)
ave2.insert(loc=0, column='datetime', value='2019-10-27 01:30:00')
ins2 = insert_row(14355, ins2, ave2)

In [35]:
ins2.iloc[4273:4277]

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
4273,2019-03-31 00:30:00,48.10000,496.9227,28422.0,25280.0,26111.0,26381.0,0.0,59.296,795.8310,23905.0,0.0,2227.3230,1370.499
4274,2019-03-31 01:00:00,48.01905,454.3874,28210.0,25066.5,26027.0,26380.5,0.0,749.875,1173.8025,23769.0,0.0,2355.1155,1349.481
4275,2019-03-31 01:30:00,48.01905,454.3874,28210.0,25066.5,26027.0,26380.5,0.0,749.875,1173.8025,23769.0,0.0,2355.1155,1349.481
4276,2019-03-31 02:00:00,47.93810,411.8521,27998.0,24853.0,25943.0,26380.0,0.0,1440.454,1551.7740,23633.0,0.0,2482.9080,1328.463


In [37]:
ins2.iloc[14353:14358]

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
14353,2019-10-27 00:30:00,-68.97,-902.93650,25071.0,21718.0,23840.0,24475.0,0.0,152.484,1895.443,24888.0,0.0,6400.315,6392.602
14354,2019-10-27 01:00:00,-35.00,-807.61505,24704.5,23683.0,23712.0,24412.0,0.0,151.615,1930.092,24785.5,0.0,6495.704,6293.020
14355,2019-10-27 01:30:00,-29.00,-891.41355,24183.5,22945.0,23453.0,24081.5,0.0,151.840,1961.328,24687.0,0.0,6496.542,6350.449
14356,2019-10-27 02:00:00,0.00,-576.16490,22720.0,22445.0,22911.0,23512.0,0.0,4633.319,4009.640,24685.0,0.0,6411.804,6384.826
14357,2019-10-27 02:30:00,-31.85,-649.11900,22291.0,22258.0,22968.0,23344.0,0.0,4899.854,3929.742,24682.0,0.0,6411.804,6384.826


In [39]:
decimals = pd.Series([2,2,0,0,0,0,0,0,0,0,0,0,0], index=['imbalancePrice','HAimbalanceVolume',
                                                         'HAload','loadForecast','HAdemand','demandForecast',
                                                         'HAsolar','HAwindOff','HAwindOn',
                                                         'aggrForecast','solarForecast','windOffForecast','windOnForecast'])
df_rounded = ins2.round(decimals)

In [46]:
cols = ['HAload','loadForecast','HAdemand','demandForecast',
        'HAsolar','HAwindOff','HAwindOn',
        'aggrForecast','solarForecast','windOffForecast','windOnForecast']
df_rounded[cols] = df_rounded[cols].applymap(np.int64)

In [48]:
df_rounded.to_csv('merged_2019.csv',index=False)

# 2018

In [53]:
df1 = pd.read_csv('BMRS_2018/Imbalance_Price_2018.csv')
df2 = pd.read_csv('BMRS_2018/Imbalance_Volume_2018.csv')

df3 = pd.read_csv('BMRS_2018/Actual_Load_2018.csv')
df4 = pd.read_csv('BMRS_2018/DA_Load_Forecast_2018.csv')

df5 = pd.read_csv('BMRS_2018/Demand_Outturn_2018.csv')
df6 = pd.read_csv('BMRS_2018/DA_Demand_Forecast_2018.csv')

df7 = pd.read_csv('BMRS_2018/Actual_Solar_Generation_2018.csv')
df8 = pd.read_csv('BMRS_2018/Actual_Wind_Offshore_Generation_2018.csv')
df9 = pd.read_csv('BMRS_2018/Actual_Wind_Onshore_Generation_2018.csv')

df10 = pd.read_csv('BMRS_2018/DA_Aggregated_Generation_Forecast_2018.csv')
df11 = pd.read_csv('BMRS_2018/DA_Solar_Generation_Forecast_2018.csv')
df12 = pd.read_csv('BMRS_2018/DA_Wind_Offshore_Generation_Forecast_2018.csv')
df13 = pd.read_csv('BMRS_2018/DA_Wind_Onshore_Generation_Forecast_2018.csv')

In [66]:
df13.drop_duplicates(subset=['local_datetime', 'settlementPeriod'])

,local_datetime,settlementPeriod,quantity
0,2018-11-02 00:00:00+00:00,1,3825.760
1,2018-11-02 00:30:00+00:00,2,3825.760
2,2018-11-02 01:00:00+00:00,3,3933.543
3,2018-11-02 01:30:00+00:00,4,3933.543
4,2018-11-02 02:00:00+00:00,5,4055.365
...,...,...,...
2875,2018-12-31 21:30:00+00:00,44,7901.926
2876,2018-12-31 22:00:00+00:00,45,7905.174
2877,2018-12-31 22:30:00+00:00,46,7905.174
2878,2018-12-31 23:00:00+00:00,47,7905.429


In [67]:
dfs = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13]
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['local_datetime','settlementPeriod'],how='inner'), dfs)
df_merged

,local_datetime,settlementPeriod,imbalancePriceAmountGBP,imbalanceQuantityMAW,quantity_x,quantity_y,demand_x,demand_y,quantity_x,quantity_y,quantity_x,quantity_y,quantity_x,quantity_y,quantity
0,2018-11-02 00:00:00+00:00,1,40.20,-260.7741,30177.0,27902,28100.0,28048.0,0.0,2102.914,2994.198,28525,0,2956.987,3825.760
1,2018-11-02 00:30:00+00:00,2,47.65,-188.4287,30487.0,27756,28282.0,27997.0,0.0,2324.970,3055.414,28488,0,2956.987,3825.760
2,2018-11-02 01:00:00+00:00,3,50.93,-73.8360,29433.0,27241,28458.0,28421.0,0.0,2200.773,3063.203,28450,0,2885.796,3933.543
3,2018-11-02 01:30:00+00:00,4,50.77,-61.5034,28317.0,26610,28477.0,28435.0,0.0,2251.779,2986.697,28395,0,2885.796,3933.543
4,2018-11-02 02:00:00+00:00,5,52.34,-78.2572,27462.0,26301,28376.0,28369.0,0.0,2002.263,3050.287,28064,0,2846.556,4055.365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2018-12-31 21:30:00+00:00,44,40.10,-880.3353,30672.0,28576,28301.0,29156.0,0.0,2952.542,6360.033,33201,0,3350.512,7901.926
2876,2018-12-31 22:00:00+00:00,45,40.00,-562.3883,27691.0,27804,27437.0,28315.0,0.0,3032.464,6466.579,32759,0,3479.663,7905.174
2877,2018-12-31 22:30:00+00:00,46,40.00,-601.8658,27030.0,27123,26858.0,27881.0,0.0,3008.399,6214.767,32111,0,3479.663,7905.174
2878,2018-12-31 23:00:00+00:00,47,40.00,-694.6639,26429.0,26418,26110.0,27188.0,0.0,3085.224,6268.369,30481,0,3665.177,7905.429


In [68]:
df_merged.columns = ['datetime','settlementPeriod','imbalancePrice','HAimbalanceVolume',
                     'HAload','loadForecast','HAdemand','demandForecast',
                     'HAsolar','HAwindOff','HAwindOn',
                     'aggrForecast','solarForecast','windOffForecast','windOnForecast']
df_merged['datetime'] = df_merged['datetime'].str[:-6]
df_merged.drop('settlementPeriod', axis=1, inplace=True)
df_merged.head()

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
0,2018-11-02 00:00:00,40.20,-260.7741,30177.0,27902,28100.0,28048.0,0.0,2102.914,2994.198,28525,0,2956.987,3825.760
1,2018-11-02 00:30:00,47.65,-188.4287,30487.0,27756,28282.0,27997.0,0.0,2324.970,3055.414,28488,0,2956.987,3825.760
2,2018-11-02 01:00:00,50.93,-73.8360,29433.0,27241,28458.0,28421.0,0.0,2200.773,3063.203,28450,0,2885.796,3933.543
3,2018-11-02 01:30:00,50.77,-61.5034,28317.0,26610,28477.0,28435.0,0.0,2251.779,2986.697,28395,0,2885.796,3933.543
4,2018-11-02 02:00:00,52.34,-78.2572,27462.0,26301,28376.0,28369.0,0.0,2002.263,3050.287,28064,0,2846.556,4055.365


In [69]:
decimals = pd.Series([2,2,0,0,0,0,0,0,0,0,0,0,0], index=['imbalancePrice','HAimbalanceVolume',
                                                         'HAload','loadForecast','HAdemand','demandForecast',
                                                         'HAsolar','HAwindOff','HAwindOn',
                                                         'aggrForecast','solarForecast','windOffForecast','windOnForecast'])
df_rounded = df_merged.round(decimals)

In [70]:
cols = ['HAload','loadForecast','HAdemand','demandForecast',
        'HAsolar','HAwindOff','HAwindOn',
        'aggrForecast','solarForecast','windOffForecast','windOnForecast']
df_rounded[cols] = df_rounded[cols].applymap(np.int64)

In [72]:
df_rounded.to_csv('merged_2018.csv',index=False)

# 2021

In [73]:
df1 = pd.read_csv('BMRS_2021/Imbalance_Price_2021.csv')
df2 = pd.read_csv('BMRS_2021/Imbalance_Volume_2021.csv')

df3 = pd.read_csv('BMRS_2021/Actual_Load_2021.csv')
df4 = pd.read_csv('BMRS_2021/DA_Load_Forecast_2021.csv')

df5 = pd.read_csv('BMRS_2021/Demand_Outturn_2021.csv')
df6 = pd.read_csv('BMRS_2021/DA_Demand_Forecast_2021.csv')

df7 = pd.read_csv('BMRS_2021/Actual_Solar_Generation_2021.csv')
df8 = pd.read_csv('BMRS_2021/Actual_Wind_Offshore_Generation_2021.csv')
df9 = pd.read_csv('BMRS_2021/Actual_Wind_Onshore_Generation_2021.csv')

df10 = pd.read_csv('BMRS_2021/DA_Aggregated_Generation_Forecast_2021.csv')
df11 = pd.read_csv('BMRS_2021/DA_Solar_Generation_Forecast_2021.csv')
df12 = pd.read_csv('BMRS_2021/DA_Wind_Offshore_Generation_Forecast_2021.csv')
df13 = pd.read_csv('BMRS_2021/DA_Wind_Onshore_Generation_Forecast_2021.csv')

In [74]:
dfs = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13]
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['local_datetime','settlementPeriod'],how='inner'), dfs)
df_merged

,local_datetime,settlementPeriod,imbalancePriceAmountGBP,imbalanceQuantityMAW,quantity_y_x,quantity_y_y,demand_x,demand_y,quantity_x,quantity_y,quantity_x,quantity_y,quantity_x,quantity_y,quantity
0,2021-01-01 00:00:00+00:00,1,73.15,843.6931,32850.0,30006,28880.0,29530.0,0,1588.649,3146.039,25357,0,2082.297,3300.832
1,2021-01-01 00:30:00+00:00,2,73.15,934.5820,33109.0,29911,29018.0,29560.0,1,1451.371,3072.821,25375,0,2082.297,3300.832
2,2021-01-01 01:00:00+00:00,3,72.00,604.7328,32383.0,29209,28252.0,29127.0,1,1403.168,3019.869,25187,0,2213.217,3282.530
3,2021-01-01 01:30:00+00:00,4,71.90,307.3387,31335.0,28041,27625.0,28456.0,0,1425.039,2884.782,25035,0,2213.217,3282.530
4,2021-01-01 02:00:00+00:00,5,71.50,48.8845,30048.0,27243,27051.0,27988.0,0,1487.909,2818.437,25095,0,2292.820,3247.227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,2021-04-30 21:30:00+01:00,44,120.00,224.0560,32721.0,31810,31881.0,31739.0,2,296.527,500.312,28934,0,3752.310,1228.446
5754,2021-04-30 22:00:00+01:00,45,135.00,315.6673,31634.0,30150,30797.0,30733.0,1,322.703,467.816,28125,0,4152.349,1233.755
5755,2021-04-30 22:30:00+01:00,46,90.00,165.2474,30513.0,28291,29688.0,29306.0,0,344.614,442.129,27377,0,4152.349,1233.755
5756,2021-04-30 23:00:00+01:00,47,85.00,261.7062,29152.0,26626,28293.0,27831.0,1,348.617,399.810,24024,0,3920.856,1203.210


In [75]:
df_merged.columns = ['datetime','settlementPeriod','imbalancePrice','HAimbalanceVolume',
                     'HAload','loadForecast','HAdemand','demandForecast',
                     'HAsolar','HAwindOff','HAwindOn',
                     'aggrForecast','solarForecast','windOffForecast','windOnForecast']
df_merged['datetime'] = df_merged['datetime'].str[:-6]
df_merged.drop('settlementPeriod', axis=1, inplace=True)
df_merged.head()

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
0,2021-01-01 00:00:00,73.15,843.6931,32850.0,30006,28880.0,29530.0,0,1588.649,3146.039,25357,0,2082.297,3300.832
1,2021-01-01 00:30:00,73.15,934.5820,33109.0,29911,29018.0,29560.0,1,1451.371,3072.821,25375,0,2082.297,3300.832
2,2021-01-01 01:00:00,72.00,604.7328,32383.0,29209,28252.0,29127.0,1,1403.168,3019.869,25187,0,2213.217,3282.530
3,2021-01-01 01:30:00,71.90,307.3387,31335.0,28041,27625.0,28456.0,0,1425.039,2884.782,25035,0,2213.217,3282.530
4,2021-01-01 02:00:00,71.50,48.8845,30048.0,27243,27051.0,27988.0,0,1487.909,2818.437,25095,0,2292.820,3247.227


In [76]:
df_merged.iloc[4128:4132]

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
4128,2021-03-28 00:00:00,39.40,239.3615,22223.0,24678,23741.0,24081.0,0,6918.636,6794.989,27565,0,9190.026,9572.874
4129,2021-03-28 00:30:00,40.10,249.2365,22761.0,24724,23988.0,23560.0,2,316.351,3970.963,27637,0,9190.026,9572.874
4130,2021-03-28 02:00:00,36.25,387.7826,22390.0,24108,23432.0,23455.0,2,6823.448,6872.973,27718,0,9274.625,9710.903
4131,2021-03-28 02:30:00,34.56,237.5914,21526.0,23437,23399.0,23414.0,0,6805.228,6881.344,27823,0,9274.625,9710.903


In [77]:
def insert_row(idx, df, df_insert):
    return df.iloc[:idx,].append(df_insert).append(df.iloc[idx:,]).reset_index(drop = True)

aveT = df_merged.iloc[4129:4131].mean(axis=0)
ave = pd.DataFrame(aveT).transpose()
ave.insert(loc=0, column='datetime', value='2021-03-28 01:00:00')
inserted = insert_row(4130, df_merged, ave)
ave['datetime'] = '2021-03-28 01:30:00'
inserted = insert_row(4131, inserted, ave)

In [78]:
inserted.iloc[4129:4133]

,datetime,imbalancePrice,HAimbalanceVolume,HAload,loadForecast,HAdemand,demandForecast,HAsolar,HAwindOff,HAwindOn,aggrForecast,solarForecast,windOffForecast,windOnForecast
4129,2021-03-28 00:30:00,40.100,249.23650,22761.0,24724.0,23988.0,23560.0,2.0,316.3510,3970.963,27637.0,0.0,9190.0260,9572.8740
4130,2021-03-28 01:00:00,38.175,318.50955,22575.5,24416.0,23710.0,23507.5,2.0,3569.8995,5421.968,27677.5,0.0,9232.3255,9641.8885
4131,2021-03-28 01:30:00,38.175,318.50955,22575.5,24416.0,23710.0,23507.5,2.0,3569.8995,5421.968,27677.5,0.0,9232.3255,9641.8885
4132,2021-03-28 02:00:00,36.250,387.78260,22390.0,24108.0,23432.0,23455.0,2.0,6823.4480,6872.973,27718.0,0.0,9274.6250,9710.9030


In [79]:
decimals = pd.Series([2,2,0,0,0,0,0,0,0,0,0,0,0], index=['imbalancePrice','HAimbalanceVolume',
                                                         'HAload','loadForecast','HAdemand','demandForecast',
                                                         'HAsolar','HAwindOff','HAwindOn',
                                                         'aggrForecast','solarForecast','windOffForecast','windOnForecast'])
df_rounded = inserted.round(decimals)

In [80]:
cols = ['HAload','loadForecast','HAdemand','demandForecast',
        'HAsolar','HAwindOff','HAwindOn',
        'aggrForecast','solarForecast','windOffForecast','windOnForecast']
df_rounded[cols] = df_rounded[cols].applymap(np.int64)

In [82]:
df_rounded.to_csv('merged_2021.csv',index=False)